# Korrelation zwischen Tweets und Aktienkursen

*Version 1.3, 02. Januar 2019*

**_Abstract_** Im Rahmen des Moduls Data Science mit Python (dsp) an der Fachhochschule Nordwestschweiz soll die Korrelation zwischen Tweets- und Aktienkursen aufgezeigt werden.

Am Beispiel des US-amerikanischen Elektorfahrzeugherstellers Tesla wurden Daten von der Social-Media-Plattform Twitter und der gehandelten Aktien des Unternehmens ausgewertet.

Aus der Analyse der Daten konnten verschiedene Schlüsse gezogen werden. Einerseits ist bei den Aktien eine Korrelation zwischen Kursdifferenz und Anzahl der gehandelten Aktien erkennbar: Die Kursdifferenzen sind an Tagen mit einem hohen gehandelten Aktienvolumen besonders gross. Die Analyse der Polarität der einzelnen Tweets lassen den Schluss zu, dass die Tweets tendenziell eher positiv ausfallen. Indem die von unterschiedlichen Quellen stammenden Datensets kombiniert wurden, konnte die Korrelation der Daten überprüft werden. Obwohl anhand von Stichproben zwar Einflüsse von Tweets auf Aktienkurse bei einzelnen Tweets erkannt wurden, konnte keine allgemeingültige Korrelation zwischen den beiden Datensets festgestellt werden.

### Table of Contents
1. [Introduction](#sec_intrtoduction)
    1. [Twitter data](#sec_twitter_data)
    2. [NASDAQ-100 data](#sec_nasdaq100_data)
2. [Results](#sec_results)
3. [Conclusion](#sec_conclusion)

### Introduction <a name="sec_introduction"></a>

Tesla ist ein US-amerikanischer Elektrofahrzeughersteller mit Sitz in Palo Alto, CA. Das Unternehmen wurde im Jahr 2003 von Elon Musk gegründet und stand in den vergangenen Jahren wiederholt in medialem Fokus (vgl. [wikipedia.org](https://de.wikipedia.org/wiki/Tesla,_Inc.)). Die Aktien sind im NASDAQ-100 enthalten. Der NASDAQ-100 ist ein Kursindex, der aus den Aktien der 100 Nicht-Finanzunternehmen in den Vereinigten Staaten mit der höchsten Marktkapitalisierung besteht.

Die Aktien von Tesla gelten als volatil und insbesondere Gründer Elon Musk nutzt Twitter als primäres Sprachrohr (vgl. [cnbc.com](https://www.cnbc.com/2018/10/05/tesla-shares-drop-nearly-5percent-after-musk-mocks-sec-on-twitter.html)). Im August 2018 beispielweise kündete Musk via Twitter an, dass er erwäge Tesla von der Börse zu nehmen. Die Ankündigung liess den Aktienkurs in den nachfolgenden Stunden ansteigen und führte letztlich dazu, dass der Handel der Tesla-Aktien zwischenzeitlich ausgesetzt wurde (Bild: [ogilvy.com](https://www.ogilvy.com/feed/11-tweets-that-turned-the-stock-market-upside-down/)):

![intro_musk.png](resources/intro_musk.png)

In der vorliegenden Data Story soll nun aufgezeigt werden, inwiefern tatsächlich eine Korrelation zwischen Twitter-Nachrichten (sog. "Tweets") und Aktienkursen festgestellt werden kann.

Grundlage für die Analysen bilden zwei separate Datensätze - diese werden in den nachfolgenden Abschnitten erläutert. Die einzelnen Schritte der Aufbereitung werden im Notebook *DataStory_Appendix* im **_Appendix A1: Data processing_** ausführlich beschrieben.

#### Twitter Daten <a name="sec_twitter_data"></a>

Twitter ist ein soziales Netwerk, wo Benutzer in kurzen Nachrichten (sog. "Tweets") miteinander kommunizieren. Twitter stellt eine eigene API zur Verfügung wo Daten konsumiert werden können. Der nachstehend verwendete Datensatz umfasst 6465 Tweets, welche entweder von den Benutzern *Tesla* oder *Elon Musk* geteilt oder mit einer @-Annotation an diese gerichtet wurden.

In [ ]:
!head -15 data/twitterdata.csv

Bevor mit den Daten gearbeitet wird, werden diese aufbereitet und in der gewünschten Form in DataFrames geladen. Die dafür notwendigen Funktionen sind im Modul *DataHelper* implementiert.

Um Aussagen über den Inhalt zu machen, werden die einzelnen Tweet analysiert. Mithilfe der `TextBlob`-Library wird für jeden Tweet ein Polaritäts-Wert berechnet. Die Polarität beschreibt, wie positiv respektive negativ eine Nachricht einzuschätzen ist (vgl. https://en.wikipedia.org/wiki/Sentiment_analysis). Dazu wird das DataFrame um eine Variable ergänzt, welche Werte im Bereich von $-1.0 \dots, 1.0$ annehmen kann. Der Wert $-1.0$ steht für eine negative Haltung gegenüber dem Empfänger der Nachricht, der Wert $1.0$ für eine positive Haltung.

In [ ]:
from util.data_helper import DataHelper as dataHelper

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

from IPython.display import display, HTML
import warnings; warnings.simplefilter('ignore') # Warninigs verified

# Prepare Twitter data
tweets = dataHelper.prepare_tweets(dataHelper)

# Display results
display(tweets)

Wir erhalten ein DatetimeIndex-indexiertes (\*) DataFrame mit den nachfolgend beschriebenen Variabeln. Eine Beschreibung der Datentypen ist im Notebook *DataStory_Appendix* im **_Appendix B1: Datatypes_** vorhanden.

* **date\***: Timestamp when the tweet was published   
* **tweet**: Twitter message, lowercased and all special characters removed
* **length**: Length of the twitter message
* **id**: Unique hash identifier, to identify twitter message
* **source**: Channel from which twitter message was sent (s. **_Appendix B2: Categorical variable "source"_**)
* **likes**: Number of likes
* **retweets**: Number of retweets
* **user**: Twitter user with whom the tweet is connected (Tesla as company or Elon Musk as founder)
* **polarity**: Polarity of twitter message in range -1..1 

#### NASDAQ-100 Daten <a name="sec_nasdaq100_data"></a>

Der NASDAQ-100 ist ein Aktienindex. Er besteht aus den Aktien der 100 Nicht-Finanzunternehmen mit der höchsten Marktkapitalisierung in den Vereinigten Staaten. Yahoo stellt eine API zum Herunterladen von Kursdaten zur Verfügung. Der nachfolgende Datensatz umfasst die Kursinformationen des Unternehmens Tesla aus dem Jahr 2018.

In [ ]:
!head -16 data/yahoodata.csv

Bevor mit den Daten gearbeitet wird, werden diese aufbereitet und in der gewünschten Form in Dataframes geladen. Die notwendigen Funktionen dafür finden sich wiederum im Modul DataHelper.

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# Prepare Yahoo data
stocks = dataHelper.prepare_stocks(dataHelper)

# Display results
display(stocks)

Wir erhalten ein DatetimeIndex-indexiertes (\*) DataFrame mit den nachfolgend beschriebenen Variabeln. Eine Beschreibung der Datentypen ist im Notebook *DataStory_Appendix* im **_Appendix B1: Datatypes_** vorhanden.

* **date\***: Trading day  
* **highest_price_usd**: Highest price of stock on this day (USD)
* **lowest_price_usd**: Lowest price of stock on this day (USD)
* **opening_price_usd**: Opening price of stock on this day (USD)
* **closing_price_usd**: Closing price of stock on this day (USD)
* **volume**: Number of stocks traded on this day
* **daily_difference_usd**: Difference between daily opening and daily closing price (USD)
* **daily_difference_usd_abs**: Absolute value of daily_difference_usd (USD)

Um Analysen über beide Datensets machen zu können, wird ein drittes, kombiniertes DataFrame erstellt. Der Twitter-Datensatz liegt in einer Granularität von Sekunden vor, während die NASDAQ100-Daten nur auf täglicher Basis vorhanden sind. Um das kombinierte DataFrame zu erstellen wird deshalb der Twitter-Datensatz resampled. Der Resampling-Vorgang und die verwendeten Aggregatsfunktionen sind im Notebook *DataStory_Appendix* im **_Appendix A1: Data processing_** beschrieben. Auf eine weitere Erläuterung und Beschreibung der Daten wird hier verzichtet.

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# Prepare combined data
resampled_tweets = dataHelper.resample_tweets(tweets)
# Join data
combined_data = dataHelper.concatenate_data(resampled_tweets, stocks)

### Results <a name="sec_results"></a>

Im folgenden Abschnitt werden die wichtigsten Erkenntnisse, welche die Daten zulassen, beschrieben. Eine Beschreibung der wichtigsten Lage- und Streuungmasse befindet sich im Notebook *DataStory_Appendix* im **_Appendix C1: Position and dispersion measures_**. Ebenfalls im Anhang wurden ausserdem die Korrelationen der verschiedenen Variabeln untersucht. Die entsprechende Analyse findet sich im Notebook *DataStory_Appendix* im **_Appendix C3: Explorative correlation overview_**. 

**1) Entwicklung des Aktienkurses über die Zeit offenbart teilweise grosse Kursdifferenzen**

Um einen Eindruck über die Entwicklung des Aktienkurses zu erhalten, wir dieser entlang einer Zeitachse geplottet. Die _Altair_-Library bietet die möglichkeit, ausserdem die Lage des Eröffnungs- und Schlusspreises im Verhältnis zum Tagestiefst- resp. Tageshöchstpreis darzustellen. Dazu wird für jeden Handelstag ein Boxplot erstellt. Die Darstellung ist äusserst informativ, aber zugegebenermassen nicht ganz einfach verständlich. Der Plot wird folgendermassen gelesen:
* Die Whisker zeigen jeweis den Tagestiefst- resp. Tageshöchstpreis
* Die Quartile stehen für Eröffnungs- resp. Schlusspreis
* Resultiert am Tagesende ein Kursgewinn ist der Boxplot grün, ansonsten rot

Indem auf dem unteren Plot ein Zeitbereich selektiert wird, wird der dargstellte Ausschnitt im oberen Plot automatisch angepasst (s. auch [Lesehilfe Plot "Entwicklung des Aktienkurses"](resources/interpretation_stocks.png)).

In [ ]:
from util.stat_helper import StatHelper as statHelper

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

import altair as alt
alt.renderers.enable('default')

# Plot stock development
statHelper.plot_stock_development(combined_data)

# Workaround for interactive charts in presentation mode
from IPython.display import IFrame
IFrame(src='resources/stock_development.html', width=950, height=640)

Falls beispielsweise der Monat August näher betrachtet wird, ist auch der eingangs erwähnte grosse Kurssprung vom  7. August gut zu erkennen. Dieser Plot erlaubt es uns, gezielt Kursverläufe an einzelnen Daten zu analysieren.

**2) Die Volatilität der gehandelten Aktien nimmt bei zunehmendem Volumen der gehandelten Aktien zu**

Der folgende Plot zeigt, dass die Kursdifferenzen bei einer grösseren Anzahl gehandelten Aktien grösser ausfällt. Für eine prägnantere Darstellung wurde für die Kursdifferenz jeweils der absolute Wert in US-Dollar verwendet.

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# Plot volum/diff correlation
statHelper.plot_stock_volatility(stocks)

# Workaround for interactive charts in presentation mode
from IPython.display import IFrame
IFrame(src='resources/stock_volatility.html', width=600, height=400)

**3) Der Inhalt der Tweets wird eher positiv bewertet**

Ganz allgemein kann festgestellt werden, dass die Tweets generell (resp. deren Polarität) als positiv eingestuft werden. Die Korrektheit der verwendeten Library wurde anhand von einzelnen Stichproben überprüft. Beim untenstehenden Plot fällt auf, dass die Tweets von Elon Musk bezüglich der Emotionalität das ganze Spektrum ausfüllen, während das Unternehmen Tesla selbst etwas neutraler twittert.

_Hinweis: Der Median bei den Tweets von Tesla befindet sich am unteren Ende der Box und ist deshalb schwer zu erkennen._

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# Plot polarity distribution
statHelper.plot_polarity_distribution(statHelper, tweets)

**4) Likes machen ein Grossteil der Aktivitäten auf Twitter aus**

Im folgenden Plot wird die Entwicklung der Twitter-Aktivitäten über die Zeit veranschaulicht. Es ist ersichtlich, dass die Twitter-Aktivitäten grösstenteils aus Likes bestehen. Der Anteil an effektiven Tweets ist innerhalb der verschiedenen Aktivitäten klein. Indem im oberen Chart ein Zeitbereich selektiert wird, werden im unteren Chart die entsprechenden Bars farblich hervorgehoben (s. auch [Lesehilfe Plot "Entwicklung Twitter-Aktivitäten"](resources/interpretation_activities.png)).

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# Plot development and ratio of twitter activities
statHelper.plot_twitter_activities(combined_data)

# Workaround for interactive charts in presentation mode
from IPython.display import IFrame
IFrame(src='resources/twitter_activities.html', width=900, height=400)

**5) Keine allgemeingültige Korrelation zwischen Aktien und Tweets feststellbar**  

Obwohl zwar Einflüsse von Tweets auf Aktienkurse bei einzelnen Tweets erkannt wurden (s. erstes Resultat), konnte keine allgemeingültige Korrelation zwischen den beiden Datensets festgestellt werden. Im untenstehenden Plot sind einige dieser nicht korrelierenden Beispiele dargestellt. Die jeweils abhängige Variable kann im Dropdown-Menü unter dem Plot ausgewählt werden (s. auch [Lesehilfe Plot "Korrelation"](resources/interpretation_correlation.png)).
Im Notebook *DataStory_Appendix* im **_Appendix C4: Quantitative correlations_** wird diese Erkenntnis noch anhand errechneter Korrelationen unterstrichen.

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF8 -*-

# Author: Marco Romanutti -- marco.romanutti@students.fhnw.ch
# https://gitlab.fhnw.ch/ML-AT-FHNW/dsp_data_stories.git
# Code of corresponding helper classes, see:
# hs2018.twitter_stock_correlation.util

# Plot correlations
statHelper.plot_diff_correlations(combined_data)

# Workaround for interactive charts in presentation mode
from IPython.display import IFrame
IFrame(src='resources/diff_correlations.html', width=900, height=430)

### Conclusion <a name="sec_conclusion"></a>

Die Analyse der Daten in den vorangegangenen Kapiteln lassen verschiedene Schlüsse zu:

* Einerseits ist bei den Aktien eine **Korrelation zwischen Kursdifferenz und Anzahl der gehandelten Aktien** erkennbar: Die Kursdifferenzen sind an Tagen mit einer hohen Anzahl gehandelten Aktien besonders gross. 
* Die Analyse der Polarität der einzelnen Tweets lassen den Schluss zu, dass die **Tweets tendenziell eher positiv** ausfallen.
* Weiter wurde festgestellt, dass ein **Grossteil der Aktivitäten auf Twitter aus Likes besteht**. Retweets und insbesondere auch eigene Posts kommen im Verhältnis dazu weniger häufig vor.
* Indem die von unterschiedlichen Quellen stammenden Datensets kombiniert wurden, konnte die Korrelation der Daten überprüft werden. Obwohl anhand von Stichproben zwar Einflüsse von Tweets auf Aktienkurse bei einzelnen Tweets vermutet werden können, konnte **keine allgemeingültige Korrelation zwischen den beiden Datensets festgestellt** werden. Um sicherzustellen, dass die fehlenden Korrelationen nicht einfach nur Tesla-spezifisch sind, wurden dieselben Analysen auf für Daten der Unternehmen Amazon und Nike durchgeführt. Die Resultate entsprachen dabei jenen vom Unternehmen Tesla - es konnten auch bei den anderen Unternehmen keine Korrelationen zwischen Aktienkursen und Tweets festgestellt werden.